<a href="https://colab.research.google.com/github/sherryfengshopify/ai-gym/blob/main/ReAct_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai

In [2]:
# @title Import and setup
from google.cloud import bigquery
from google.colab import data_table
import bigframes.pandas as bpd
import pandas as pd
from openai import OpenAI
import json
import yaml

# project = 'sdp-prd-build-ecosystem' # Project ID inserted based on the query results selected to explore
# location = 'us-central1' # Location inserted based on the query results selected to explore

llm = OpenAI(
    base_url="https://openai-proxy.shopify.ai/v1",
    api_key="shopify-eyJpZCI6IjhhOTFmMmIyYjU2MzQ3NzFlMzQyZWU2NDRjMzFmYTY3IiwibW9kZSI6InBlcnNvbmFsIiwiZW1haWwiOiJzaGVycnkuZmVuZ0BzaG9waWZ5LmNvbSIsImV4cGlyeSI6MTczMzQyODAzNX0=-PJz6nHOe6gFKKQzAWyX8ySg5WzQ6llYcfcTFSKJiOJM="
)

# client = bigquery.Client(project=project, location='US')
# data_table.enable_dataframe_formatter()

In [4]:
from google.colab import auth
auth.authenticate_user()

In [5]:
import openai
import re
import httpx
import os

In [8]:
# client = OpenAI()

MODEL = 'gpt-4o-mini'
prompt = 'Write something short but funny.'

chat_completion = llm.chat.completions.create(
    model=MODEL,
    messages=[
        {
            'role': 'user',
            'content': prompt,
        }
    ]
)

In [9]:
chat_completion.choices[0].message.content

'Why did the scarecrow win an award? Because he was outstanding in his field!'